In [46]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [47]:
# URLs des catégories à scraper
CATEGORIES = {
    "emploi": "https://www.expat-dakar.com/emploi",
    "terrains_a_vendre": "https://www.expat-dakar.com/terrains-a-vendre",
   # "accessoires_multimedia": "https://www.expat-dakar.com/multimedia",
   # "decoration_linge_de_maison": "https://www.expat-dakar.com/decoration-linge-de-maison"
   # "véhicules": "https://www.expat-dakar.com/vehicules"
   # "maison": "https://www.expat-dakar.com/maison"
}


In [48]:
# dossier pour stocker les images
os.makedirs("images", exist_ok=True)

In [49]:
# F de télécharger les images
def download_image(url, filename):
    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(filename, 'wb') as f:
                f.write(response.content)
    except Exception as e:
        print(f"Erreur lors du téléchargement de l'image : {e}")

In [50]:
# Fonction pour scraper une catégorie
def scrape_category(category_name, category_url):
    page = 1
    data = []

    while True:
        print(f"Scraping {category_name}, page {page}...")
        url = f"{category_url}?page={page}"
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        # chercher toutes les annonces sur la page
        listings = soup.find_all("div", class_="listings-cards__list-item")
        if not listings:
            break

        for listing in listings:
            try:
                # Extraction titre
                title_element = listing.find("div", class_="listing-card__header__title")
                title = title_element.text.strip() if title_element else "Non spécifié"

                # Extraction prix
                price_element = listing.find("span", class_="listing-card__price__value")
                price = price_element.text.strip() if price_element else "Non spécifié"

                # Extraction de la loca
                location_element = listing.find("div", class_="listing-card__header__location")
                location = location_element.text.strip() if location_element else "Non spécifié"

                # Extraire l'état du produit
                condition_element = listing.find("span", class_="listing-card__header__tags__item--condition")
                condition = condition_element.text.strip() if condition_element else "Non spécifié"

                # Extraction dex image
                image_element = listing.find("img", class_="listing-card__image__resource")
                image_url = image_element["src"] if image_element else None

                # Nommation et télécharger desimages
                image_filename = f"images/{category_name}_{page}_{listings.index(listing)}.jpg"
                if image_url:
                    download_image(image_url, image_filename)

                # Ajouter les données à la liste
                data.append({
                    "Catégorie": category_name,
                    "Titre": title,
                    "Prix": price,
                    "Localisation": location,
                    "État": condition,
                    "Image": image_filename if image_url else "Aucune image"
                })
            except Exception as e:
                print(f"Erreur lors de l'extraction des données d'une annonce : {e}")

        page += 1

    return data

In [51]:
# Scrapeur de toutes les catégories
all_data = []
for category_name, category_url in CATEGORIES.items():
    all_data.extend(scrape_category(category_name, category_url))
    
# Sauvegarde des données 
df = pd.DataFrame(all_data)
df.to_excel("annonces_categories.xlsx", index=False)

print("Scraping terminé.")


Scraping emploi, page 1...
Scraping emploi, page 2...
Scraping emploi, page 3...
Scraping emploi, page 4...
Scraping emploi, page 5...
Scraping emploi, page 6...
Scraping emploi, page 7...
Scraping emploi, page 8...
Scraping emploi, page 9...
Scraping emploi, page 10...
Scraping emploi, page 11...
Scraping emploi, page 12...
Scraping emploi, page 13...
Scraping emploi, page 14...
Scraping emploi, page 15...
Scraping emploi, page 16...
Scraping emploi, page 17...
Scraping emploi, page 18...
Scraping emploi, page 19...
Scraping emploi, page 20...
Scraping terrains_a_vendre, page 1...
Scraping terrains_a_vendre, page 2...
Scraping terrains_a_vendre, page 3...
Scraping terrains_a_vendre, page 4...
Scraping terrains_a_vendre, page 5...
Scraping terrains_a_vendre, page 6...
Scraping terrains_a_vendre, page 7...
Scraping terrains_a_vendre, page 8...
Scraping terrains_a_vendre, page 9...
Scraping terrains_a_vendre, page 10...
Scraping terrains_a_vendre, page 11...
Scraping terrains_a_vendre, p